In [1]:
%load_ext autoreload
%autoreload 2
    

# Running MD simulations using nff and ASE

This Jupyter Notebook shows how the `nff` package interfaces with the Atomistic Simulation Environment (ASE). We assume the user went through tutorial `01_training`, so we can load the pretrained models without having to train them again.

As before, importing the dependencies:

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from ase import Atoms
from ase.md.verlet import VelocityVerlet

from nff.md.nve import Dynamics
from nff.data import Dataset
from nff.train import load_model, evaluate
import nff.utils.constants as const
from ase import units
from nff.io import NeuralFF, AtomsBatch

## Loading the relevant data

We reload the dataset and create a `GraphLoader` as we did last time:

In [3]:
dataset = Dataset.from_file('data/dataset.pth.tar')


### Creating Atoms

As before, we can create an `Atoms` object from any element of the dataset. Let's take the first one, for simplicity:

In [4]:
props = dataset[0].copy()
atoms = AtomsBatch(positions=props['nxyz'][:, 1:], 
                   numbers=props['nxyz'][:, 0], 
                   props=props,
                   )

### Creating the ASE calculator

Now we just have to load the ASE calculator from a pretrained model. One way of doing so is through the in-build `from_file` method. You just have to specify the folder where the model was trained and subsequently stored.

In [5]:
nff_ase = NeuralFF.from_file('sandbox/', device=0)

/home/saxelrod/miniconda3/envs/htvs/lib/python3.7/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'nff.nn.models.schnet.SchNet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/saxelrod/miniconda3/envs/htvs/lib/python3.7/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/saxelrod/miniconda3/envs/htvs/lib/python3.7/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. you can retrieve 

Assigning this calculator to `atoms` is easy:

In [6]:
atoms.set_calculator(nff_ase)

### Configuring the dynamics for the system

In this example, we will run an NVE dynamics simulation. We will use the default parameters there implemented to run a trajectory for an ethanol molecule. The parameters we will specify are the following:

* `T_init`: initial temperature of the simulation
* `time_step`: time step in femtoseconds
* `thermostat`: ASE integrator to use when performing the simulation
* `thermostat_params`: keyword arguments for ase.Integrator class, will be different case-by-case
* `steps`: number of steps to simulate
* `save_frequency`: how often (in steps) save the pose of the molecule in a file
* `nbr_list_update_freq`: how often (in steps) to update the neighbor list (not yet implemented)
* `thermo_filename`: output file for the thermodynamics log
* `traj_filename`: output file for the ASE trajectory file
* `skip`: number of initial frames to skip when recording the trajectory

In [7]:
md_params = {
    'T_init': 450,
    'time_step': 0.5,
#     'thermostat': NoseHoover,   # or Langevin or NPT or NVT or Thermodynamic Integration
#     'thermostat_params': {'timestep': 0.5 * units.fs, "temperature": 120.0 * units.kB,  "ttime": 20.0}
    'thermostat': VelocityVerlet,  
    'thermostat_params': {'timestep': 0.5 * units.fs},
    'steps': 200,
    'save_frequency': 10,
    'nbr_list_update_freq': 20,
    'thermo_filename': 'thermo.log',
    'traj_filename': 'atoms.traj',
    'skip': 0
}

In [8]:
nve = Dynamics(atoms, md_params)
nve.run()

Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]


/home/saxelrod/miniconda3/envs/htvs/lib/python3.7/site-packages/ase/atoms.py:921: VisibleDeprecationWarning: Use get_global_number_of_atoms() instead
  np.VisibleDeprecationWarning)
/home/saxelrod/Repo/projects/master/NeuralForceField/nff/nn/utils.py:121: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729138878/work/torch/csrc/utils/python_arg_parser.cpp:882.)
  nbr_list = mask.nonzero()


0.0000           0.3158      -0.1916       0.5074   436.2

0.0050           0.3154      -0.0785       0.3938   338.5

0.0100           0.3148      -0.1335       0.4483   385.4

0.0150           0.3168       0.0045       0.3123   268.4

0.0200           0.3163      -0.2513       0.5677   488.0

0.0250           0.3169      -0.0423       0.3592   308.8

0.0300           0.3151      -0.4316       0.7468   641.9

0.0350           0.3147      -0.3032       0.6179   531.1

0.0400           0.3140      -0.2451       0.5591   480.6

0.0450           0.3144      -0.2008       0.5152   442.9

0.0500           0.3158      -0.0040       0.3198   274.9

0.0550           0.3161      -0.2534       0.5695   489.5

0.0600           0.3172       0.0941       0.2231   191.8

0.0650           0.3147      -0.2455       0.5602   481.5

0.0700           0.3158      -0.1851       0.5009   430.6

0.0750           0.3158      -0.2395       0.5554   477.4

0.0800           0.3166      -0.1315       0.4481   385.

### Models with directed neighbor lists

The default assumes that you're using SchNet, which uses an undirected neighbor list to save memory. If you're using Painn, DimeNet, or any model with directional information, you will need a directed neighbor list. If you don't specify this then you will get an error. 

While these models automatically make any neighbor list into a directed one when called, the same is not true of periodic boundary conditions. The dataest offsets from the PBC depend on whether the dataset neighbor list is directed or not.

In [9]:
props = dataset[0].copy()
atoms = AtomsBatch(positions=props['nxyz'][:, 1:], 
                   numbers=props['nxyz'][:, 0], 
                   props=props,
                   )
nff_ase = NeuralFF.from_file('sandbox_painn/', device=0)
atoms.set_calculator(nff_ase)

try:
    nve = Dynamics(atoms, md_params)
    nve.run()
except Exception as e:
    print(e)

Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]
Painn needs a directed neighbor list


If you do then you will be fine!

In [10]:
props = dataset[0].copy()
atoms = AtomsBatch(positions=props['nxyz'][:, 1:], 
                   numbers=props['nxyz'][:, 0], 
                   props=props,
                   directed=True)
nff_ase = NeuralFF.from_file('sandbox_painn/', device=0)
atoms.set_calculator(nff_ase)

nve = Dynamics(atoms, md_params)
nve.run()

Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]
0.0000           0.2408      -0.1615       0.4024   345.9

0.0050           0.2394      -0.2874       0.5268   452.8

0.0100           0.2397      -0.1538       0.3935   338.2

0.0150           0.2403      -0.1675       0.4078   350.6

0.0200           0.2408      -0.3277       0.5685   488.7

0.0250           0.2417      -0.1517       0.3934   338.2

0.0300           0.2414      -0.1296       0.3710   318.9

0.0350           0.2412      -0.1576       0.3988   342.8

0.0400           0.2398      -0.2804       0.5202   447.1

0.0450           0.2401      -0.0993       0.3394   291.7

0.0500           0.2406      -0.2275       0.4682   402.4

0.0550           0.2401      -0.3081       0.5482   471.3

0.0600           0.2408      -0.1992       0.4400   378.2

0.0650           0.2418      -0.1652       0.4070   349.9

0.0700           0.2416       0.0180       0.2236   192.2

0.0750           0.2405      -0.1214       0.3619   311.1


The dynamics conserved the energy. The temperature varied throughout the simulation, as expected.

# Additional properties and options

You can also specify the properties that you want the calculator to predict. The default is `energy` and `forces`, but you can also add `stress`.

However, if you run this for ethanol you will get an error, because it doesn't have any lattice vectors:

In [11]:
nff_ase.properties = ['energy', 'forces', 'stress']
atoms.set_calculator(nff_ase)

try:
    nve = Dynamics(atoms, md_params)
    nve.run()
except Exception as e:
    print(e)

Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]      ---------------------- stress [GPa] -----------------------
You have 0 lattice vectors: volume not defined


If we make an artificial unit cell, it all works fine:

In [12]:
nff_ase.properties = ['energy', 'forces', 'stress']
atoms.set_calculator(nff_ase)
atoms.set_cell(2 * np.identity(3))
nve = Dynamics(atoms, md_params)
nve.run()



Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]      ---------------------- stress [GPa] -----------------------
0.0000           0.4442      -0.3457       0.7898   678.9     -0.871     26.839    -23.691    -27.217    -33.388    -29.995

0.0050           0.4477       0.0219       0.4258   366.1      2.117     64.619     43.543     27.383      5.637    -40.618

0.0100           0.4478      -0.0412       0.4891   420.4   -106.245    -68.407    -38.358    -39.280      7.591     63.665

0.0150           0.4470       0.0927       0.3543   304.5    -23.734    -34.153     39.779     35.955    -45.249    -20.279

0.0200           0.4474      -0.0062       0.4536   390.0    -18.387    -23.564    -11.814    -38.303     39.612     13.974

0.0250           0.4458       0.1687       0.2771   238.2    146.343     47.266     23.320     27.814     17.208    -47.881

0.0300           0.4443       0.0117       0.4326   371.9     45.044     38.673     -1.273    -35.237     34.992     19.666

0.0

Since `stress` is an implemented property, we can get the stress on the atoms:

In [13]:
atoms.get_stress()

array([ 0.17091832, -0.06599259, -0.10472327,  0.2841331 , -0.09786777,
        0.3425693 ], dtype=float32)

You can additionally request larger offsets for periodic structures than the default. To do so, use `requires_large_offsets=True` when making the atoms:

In [14]:
lattice = [[1.155363155, 0.0, 0.0], 
           [0.0, 1.155363155, 0.0], 
           [0.0, 0.0, 1.07085842]]

# default of `requires_large_offsets` is False
old_atoms = AtomsBatch(positions=props['nxyz'][:, 1:], 
                       numbers=props['nxyz'][:, 0], 
                       props=props,
                       cell=lattice,
                       pbc=True
                       )

new_atoms = AtomsBatch(positions=props['nxyz'][:, 1:], 
                       numbers=props['nxyz'][:, 0], 
                       props=props,
                       requires_large_offsets=True,
                       cell=lattice,
                       pbc=True
                       )

If we compare the offsets in the new and old `atoms` objects, we see that there are differences:

In [15]:
old_nbrs, old_offsets = old_atoms.update_nbr_list()
new_nbrs, new_offsets = new_atoms.update_nbr_list()

print((old_offsets.to_dense() == new_offsets.to_dense()).all().item())
print(old_offsets)
print(new_offsets)


False
tensor(indices=tensor([[ 0,  0,  1,  1,  2,  3,  3,  4,  5,  5,  5,  6,  6,  6,
                         7,  7,  8,  9,  9,  9, 10, 10, 11, 12, 13, 13, 14, 14,
                        15, 15, 15, 16, 16, 17, 18, 18, 19, 19, 19, 20, 21, 22,
                        22, 22, 23, 23, 24, 24, 24, 25, 25, 25, 26, 26, 27, 27,
                        27, 28, 28, 29, 29, 29, 30, 30, 30, 31, 31, 31, 32, 32,
                        33, 34, 34, 35, 35],
                       [ 0,  1,  0,  1,  2,  1,  2,  0,  0,  1,  2,  0,  1,  2,
                         0,  1,  0,  0,  1,  2,  0,  1,  0,  2,  1,  2,  0,  1,
                         0,  1,  2,  0,  1,  0,  0,  2,  0,  1,  2,  1,  2,  0,
                         1,  2,  0,  1,  0,  1,  2,  0,  1,  2,  0,  1,  0,  1,
                         2,  0,  1,  0,  1,  2,  0,  1,  2,  0,  1,  2,  0,  1,
                         2,  0,  2,  0,  2]]),
       values=tensor([ 1.1554,  1.1554, -1.1554,  1.1554,  1.0709, -1.1554,
                      -1.0

## Visualizing the trajectory

To visualize the trajectory in this Jupyter Notebook, you will have to install the package [nglview](https://github.com/arose/nglview).

In [16]:
import nglview as nv
from ase.io import Trajectory

Displaying the trajectory:

In [17]:
%matplotlib notebook

traj = Trajectory('atoms.traj')
nv.show_asetraj(traj)

NGLWidget(max_frame=20)

Looks like the atoms are still together. Visual inspection says that the trajectory is reasonable. Yay for `nff`!